## Tutorial: MCMC

In [ ]:
#| hide
#skip
! [ -e /content ] && pip install -Uqq pyndamics3 emcee # upgrade pyndamics3 on colab

In [ ]:
%matplotlib inline
from pylab import *

In [ ]:
from sie.mcmc import StatsModel

In [ ]:
xy1="""
X	Y
10	8.04
8	6.95
13	7.58
9	8.81
11	8.33
14	9.96
6	7.24
4	4.26
12	10.84
7	4.82
5	5.68
"""
xy=xy1
x,y=array([_.split() for _ in xy.strip().split('\n')[1:]],dtype=float).T
x,y

(array([10.,  8., 13.,  9., 11., 14.,  6.,  4., 12.,  7.,  5.]),
 array([ 8.04,  6.95,  7.58,  8.81,  8.33,  9.96,  7.24,  4.26, 10.84,
         4.82,  5.68]))

In [ ]:
model=StatsModel()
model.add_data(x=x,y=y)
model.add("m ~ Normal(0,10)")
model.add("b ~ Normal(0,10)")
model.add("σ ~ Jeffreys()")
model.add("res ~ Normal(m*x+b-y,σ,sum=True)")
model.initialize()
model

KeyError: '_lnprior'

In [ ]:
print(model.function_str)

def _init_prior(nwalkers,ndim,slices,extra={}):

    _value=None

    return _value


def _lnlikelihood(θ,data,slices,extra={}):
    x=data['x']
    y=data['y']

    m=θ[slices.m]
    b=θ[slices.b]
    σ=θ[slices.σ]

    res=θ[slices.res]

    _value=0

    _value+=Normal(m*x+b-y,σ,sum=True)(res)

    return _value.sum()



In [ ]:
print(model.make_func())

In [ ]:
model.run_mcmc(800,repeat=3)
model.plot_chains()

In [ ]:
model.best_estimates()

In [ ]:
model.plot_chains('σ')

In [ ]:
model.plot_distributions()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()